# Data Import

In [342]:
#importing packages

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [216]:
#importing the activity log for the desired month 

month = '12'
year = '2021'

activity = month + '-' + year + '_Activity_Log_for_Miller_Family_Medicine'

act = pd.read_excel("/Users/adam/Desktop/"+activity+".xlsx",sheet_name='All Data, ' + month + '-' + year)
act.head()

,Patient,MRN,Date of Service,Care Type,Time Spent (min),Care Provider,Billable
0,Mary J Lauer,18204,12/01/2021 09:47AM,CCM,60.62,Valerie Wan,Yes
1,Karen Anna,14944,12/01/2021 11:10AM,CCM,60.95,Valerie Wan,Yes
2,Bradley C Baldwin,17901,12/01/2021 12:38PM,CCM,29.30,Valerie Wan,Yes
3,Sean K Code,15627,12/01/2021 13:41PM,CCM,87.17,Valerie Wan,Yes
4,Joan Anna,15955,12/01/2021 13:57PM,CCM,61.47,Valerie Wan,Yes


# Staff Minutes and Salary

### Total Minutes and Salary By Employee 

In [333]:
type_key = {
'Adam  Agovino':'Pharm', 
'Amanda DeFrain':'Pharm',
'Morgan Martinson':'Intern',
'Olivia Gula':'Intern',
'Claire Meredith':'Student',
'Sydney Sadkin':'Intern',
'Valerie Wan':'Pharm'
}

pay_key_hr = {
'Pharm': 55,
'Intern': 16,
'Student':0
}

staff = act.groupby('Care Provider', as_index = False)['Time Spent (min)'].sum()
staff['Type']= staff['Care Provider'].map(type_key)
staff['Enroll'] = 0
staff['Payment'] = ((staff['Type'].map(pay_key_hr))*(staff['Time Spent (min)']))/60

jacob = 126
marley = 2.5

col = list(staff.columns)
data = [['Jacob DeFrain',0,'Other',jacob,jacob*20],
       ['Marley Sorg',0,'Other',0,marley*10]]

other = pd.DataFrame(data,columns=col)

all_staff = pd.concat([staff,other])

total_salary = all_staff['Payment'].sum()

all_staff

,Care Provider,Time Spent (min),Type,Enroll,Payment
0,Adam Agovino,410.33,Pharm,0,376.135833
1,Amanda DeFrain,2635.38,Pharm,0,2415.765000
2,Claire Meredith,1896.05,Student,0,0.000000
3,Morgan Martinson,2074.03,Intern,0,553.074667
4,Olivia Gula,605.78,Intern,0,161.541333
5,Sydney Sadkin,2184.75,Intern,0,582.600000
6,Valerie Wan,5829.07,Pharm,0,5343.314167
0,Jacob DeFrain,0.00,Other,126,2520.000000
1,Marley Sorg,0.00,Other,0,25.000000


### Total Minutes By Employee and Care Type

In [334]:
staff_by_type = act.groupby(['Care Provider','Care Type'],as_index=False)['Time Spent (min)'].sum()
staff_by_type

,Care Provider,Care Type,Time Spent (min)
0,Adam Agovino,CCM,410.33
1,Amanda DeFrain,CCM,2459.87
2,Amanda DeFrain,RPM,175.51
3,Claire Meredith,CCM,1742.64
4,Claire Meredith,RPM,153.41
5,Morgan Martinson,CCM,1834.04
6,Morgan Martinson,RPM,239.99
7,Olivia Gula,CCM,605.78
8,Sydney Sadkin,CCM,2147.27
9,Sydney Sadkin,RPM,37.48


# CCM Minutes and Reimbursement

In [349]:
new_act = act[(act['Care Type']=='CCM')]
minss = new_act['Time Spent (min)'].sum()
mins = minss.astype(str)
hr = minss/60
hrs = hr.astype(str)
minutes = 'Total Minutes Spent in Medi-CCM = '+mins
hours = 'Total Hours Spent in Medi-CCM = '+hrs + ' (This should match the number on the Medi-CCM Dashboard)'
print(minutes)
print(hours)

Total Minutes Spent in Medi-CCM = 15029.0
Total Hours Spent in Medi-CCM = 250.48333333333332 (This should match the number on the Medi-CCM Dashboard)


### Subset of Patients From Extended Dataset

In [350]:
pts = act.groupby(['Patient', 'Care Type'], as_index = False)['Time Spent (min)'].sum()

col = pts['Time Spent (min)']
ccm_conditions  = [ col >= 90, (col < 90) & (col >= 60), (col < 60) & (col >= 40), (col < 40) & (col >= 20), col < 20 ]
ccm_choices     = [ 90, 60, 40, 20, 0 ]

rpm_conditions  = [ col >= 60, (col < 60) & (col >= 40), (col < 40) & (col >= 20), col < 20 ]
rpm_choices     = [ 60, 40, 20, 0 ]

pts["Billable Time"] = np.where(pts['Care Type']=='CCM', np.select(ccm_conditions, ccm_choices, default=np.nan), np.select(rpm_conditions, rpm_choices, default=np.nan))

pts['Over_Minutes'] = np.where(pts['Billable Time']!=0, pts['Time Spent (min)']-pts['Billable Time'], 0)
pts['Over_Minutes'] = pts['Over_Minutes'].astype(float)

ccm_pts = pts[pts['Care Type']=='CCM']
rpm_pts = pts[pts['Care Type']=='RPM']

ccm_pts['CCM_Type'] = np.where((ccm_pts['Billable Time']!=90.0), 'Non-Complex', 'Complex')
rpm_pts['CCM_Type'] = 0

pts = pd.concat([ccm_pts,rpm_pts])

non_ccm_rb = 39.89
non_add = 36.46
comp = 130.49

col = pts['Billable Time']
ccm_conditions2 = [col==20.0,col==40.0,col==60.0,col==90.0]
ccm_choices2 = [(non_ccm_rb), (non_ccm_rb+non_add), (non_ccm_rb+non_add+non_add), comp]
pts['Bill Out'] = np.where(pts['Care Type']=='CCM', np.select(ccm_conditions2, ccm_choices2, default=np.nan), 0)

pts['99490'] = np.where(pts['CCM_Type']=='Non-Complex', 1, 0)

ccm_choices3 = [0,1,2,0]
pts['99439'] = np.where(pts['CCM_Type']=='Non-Complex',np.select(ccm_conditions2, ccm_choices3, default=np.nan), 0)

pts['99487'] = np.where(pts['CCM_Type']=='Complex', 1, 0)

pts['99489'] = np.where(pts['CCM_Type']=='Complex', 1, 0)

pts

,Patient,Care Type,Time Spent (min),Billable Time,Over_Minutes,CCM_Type,Bill Out,99490,99439,99487,99489
0,"A ""LaRue"" L Mullins",CCM,63.48,60.0,3.48,Non-Complex,112.81,1,2.0,0,0
1,Albert R Kephart,CCM,60.93,60.0,0.93,Non-Complex,112.81,1,2.0,0,0
2,Alice J Odonnell,CCM,100.92,90.0,10.92,Complex,130.49,0,0.0,1,1
3,Angela L Lubert,CCM,61.42,60.0,1.42,Non-Complex,112.81,1,2.0,0,0
4,Anna M Yacisin,CCM,65.05,60.0,5.05,Non-Complex,112.81,1,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
255,Ruth A Adams,RPM,0.53,0.0,0.00,0,0.00,0,0.0,0,0
259,Samuel L Reed,RPM,0.77,0.0,0.00,0,0.00,0,0.0,0,0
262,Sean K Code,RPM,53.74,40.0,13.74,0,0.00,0,0.0,0,0
273,Steven Mishock,RPM,21.62,20.0,1.62,0,0.00,0,0.0,0,0


### Summary of Minutes, Reimbursement, MD Share, and Net By Code 

In [351]:
nc_total = pts['99490'].sum() 
nc_add_on_total = pts['99439'].sum()
complex_total = pts['99487'].sum()
complex_add_on = pts['99489'].sum()

nc_min = pts['99490'].sum() * 20
nc_add_on_min = pts['99439'].sum() * 20
comp_min = pts['99487'].sum() * 60
comp_add_on_min = pts['99489'].sum() * 30

nc_reimb = 39.89
nc_add_reimb = 36.46
comp_reimb = 88.21
comp_add_reimb = 42.28

data = [['99490',nc_total,nc_min,nc_reimb], 
        ['99439',nc_add_on_total,nc_add_on_min,nc_add_reimb], 
        ['99487',complex_total,comp_min,comp_reimb], 
        ['99489',complex_add_on, comp_add_on_min, comp_add_reimb]]

summary = pd.DataFrame(data, columns = ['Code','# of Instances','Billable Minutes','Reimbursement Rate'])

summary['Total Reimbursement'] = summary['# of Instances'] * summary['Reimbursement Rate']
summary['Doctor Reimbursement'] = summary['Total Reimbursement'] * 0.25
summary['Our Reimbursement'] = summary['Total Reimbursement'] * 0.75

summary

,Code,# of Instances,Billable Minutes,Reimbursement Rate,Total Reimbursement,Doctor Reimbursement,Our Reimbursement
0,99490,206.0,4120.0,39.89,8217.34,2054.3350,6163.0050
1,99439,379.0,7580.0,36.46,13818.34,3454.5850,10363.7550
2,99487,25.0,1500.0,88.21,2205.25,551.3125,1653.9375
3,99489,25.0,750.0,42.28,1057.00,264.2500,792.7500


### Billable vs. Unbillable Time

In [360]:
total_billable_minutes = pts['Billable Time'].sum()

total_minutes = pts['Time Spent (min)'].sum()

ccm = pts[pts['Care Type']=='CCM']
ccm_bill_min = ccm['Billable Time'].sum()
ccm_tot_min = ccm['Time Spent (min)'].sum()

rpm = pts[pts['Care Type']=='RPM']
rpm_bill_min = rpm['Billable Time'].sum()
rpm_tot_min = rpm['Time Spent (min)'].sum()

total_minutes, total_billable_minutes, rpm_min

data
col = ['','Total Minutes Spent','Billable Minutes','Unbillable Hours']
data = [['CCM',ccm_tot_min,ccm_bill_min,(ccm_tot_min-ccm_bill_min)/60],
     ['RPM',rpm_tot_min,rpm_bill_min,(rpm_tot_min-rpm_bill_min)/60],
     ['Total',total_minutes,total_billable_minutes,(total_minutes-total_billable_minutes)/60]]
minutes = pd.DataFrame(data,columns=col)
minutes

,,Total Minutes Spent,Billable Minutes,Unbillable Hours
0,CCM,15029.00,13930.0,18.316667
1,RPM,606.39,400.0,3.439833
2,Total,15635.39,14330.0,21.756500


# Balance Sheet

In [336]:
ccm_billable = pts[pts['Care Type']=='CCM']
ccm_billable['Bill Out'] = ccm_billable['Bill Out'].astype(np.float16)

extra_costs = 350
ccm_total_bill = ccm_billable['Bill Out'].sum()
ccm_doc_share = ccm_total_bill * 0.25
our_gross = ccm_total_bill * 0.75
our_net = our_gross - total_salary - extra_costs

data = [['Gross Reimbursed', ccm_total_bill], ['Doctor Share', ccm_doc_share], ['Our Invoice to Doc', our_gross], ['Salary', total_salary], ['Extra Costs', extra_costs], ['Net', our_net]]

balance_sheet = pd.DataFrame(data, columns = ['','Amount'])
balance_sheet['Amount'] = balance_sheet['Amount'].apply(lambda x: "${:,.2f}".format((x)))
balance_sheet

,,Amount
0,Gross Reimbursed,"$25,264.00"
1,Doctor Share,"$6,316.00"
2,Our Invoice to Doc,"$18,948.00"
3,Salary,"$11,977.43"
4,Extra Costs,$350.00
5,Net,"$6,620.57"


# Extra Parameters

### Patients whose Minutes are close to the Next Level of Billing (and should be investigated)

In [340]:
close = pts[(pts['Over_Minutes']>10) & (pts['Care Type']=='CCM') & (pts['Billable Time']!=90.0)]
close.sort_values(by=['Over_Minutes'], ascending=False)

,Patient,Care Type,Time Spent (min),Billable Time,Over_Minutes,CCM_Type,Bill Out,99490,99439,99487,99489
188,Mary J Lauer,CCM,80.72,60.0,20.72,Non-Complex,112.81,1,2.0,0,0
171,M J Hertzog,CCM,78.99,60.0,18.99,Non-Complex,112.81,1,2.0,0,0
281,Vesta J Huber,CCM,78.80,60.0,18.80,Non-Complex,112.81,1,2.0,0,0
272,Steven Mishock,CCM,77.70,60.0,17.70,Non-Complex,112.81,1,2.0,0,0
277,Thomas Kruise,CCM,77.23,60.0,17.23,Non-Complex,112.81,1,2.0,0,0
128,Jean P Ruhe,CCM,76.08,60.0,16.08,Non-Complex,112.81,1,2.0,0,0
33,Claire Ashcroft,CCM,75.50,60.0,15.50,Non-Complex,112.81,1,2.0,0,0
140,John Schraff,CCM,35.28,20.0,15.28,Non-Complex,39.89,1,0.0,0,0
90,George A Eckenrode,CCM,75.27,60.0,15.27,Non-Complex,112.81,1,2.0,0,0
170,Lynn A Murphy,CCM,75.10,60.0,15.10,Non-Complex,112.81,1,2.0,0,0


### Patients who are below the 20 Minute Minimum Billing Threshhold 

In [338]:
alert = pts[(pts['Billable Time']==0) & (pts['Care Type'] == 'CCM')]

#final_df = df.sort_values(by=['2'], ascending=False)

alert.sort_values(by =['Time Spent (min)'], ascending=False)

,Patient,Care Type,Time Spent (min),Billable Time,Over_Minutes,CCM_Type,Bill Out,99490,99439,99487,99489
257,Ruth Semonich,CCM,11.86,0.0,0.0,Non-Complex,NaN,1,NaN,0,0
